In [1]:
import pandas as pd
import json
import folium

In [ ]:
# Read rides_list.csv generated from get_rides_list.js
df = pd.read_csv('rides_list.csv')

df['start'] = pd.to_datetime(df['start'], unit='ms')
df['end'] = pd.to_datetime(df['end'], unit='ms')

df.sort_values('end')

In [ ]:
def format_ids_for_js(df, column_name='id'):
    # Create the opening of the JavaScript declaration
    js_output = 'const routeIds = [\n'
    
    # Add each ID as a quoted string with comma and newline
    for id_value in df[column_name]:
        js_output += f"  '{id_value}',\n"
    
    # Add closing bracket
    js_output += ']'
    
    return js_output

# Prints the JavaScript declaration of routeIds to be used in routes_info.js
print(format_ids_for_js(df))

In [2]:
def plot_routes(json_data, show_start=True, show_end=True, show_route_lines=True):
    """
    Plot routes on a map with configurable markers and lines.
    
    Args:
        json_data: JSON data containing routes
        show_start (bool): Whether to show start markers
        show_end (bool): Whether to show end markers
        show_route_lines (bool): Whether to show route lines
    """
    # Parse JSON data
    data = json.loads(json_data) if isinstance(json_data, str) else json_data
    
    # Check if routes exist
    if not data.get('routes'):
        print("Warning: No routes found in data")
        return folium.Map(location=[0, 0], zoom_start=2)
    
    # Filter out routes with empty locations
    valid_routes = [route for route in data['routes'] 
                   if route.get('locations') and len(route['locations']) > 0]
    
    if not valid_routes:
        print("Warning: No valid routes with locations found")
        return folium.Map(location=[0, 0], zoom_start=2)
    
    # Create a map centered on the first valid coordinate
    first_route = valid_routes[0]
    first_location = first_route['locations'][0]
    
    map_center = [first_location['lat'], first_location['lon']]
    m = folium.Map(location=map_center, zoom_start=13)
    
    # Define colors for different routes
    colors = ['red', 'blue', 'green', 'purple', 'orange', 'darkred', 'darkblue', 'darkgreen']
    
    # Plot each valid route
    for i, route in enumerate(valid_routes):
        try:
            # Extract coordinates for the route
            coordinates = [[loc['lat'], loc['lon']] 
                         for loc in route['locations'] 
                         if 'lat' in loc and 'lon' in loc]
            
            if not coordinates:
                print(f"Warning: Route {route.get('id', i)} has no valid coordinates")
                continue
                
            # Create a PolyLine for the route if enabled
            if show_route_lines:
                color = colors[i % len(colors)]
                folium.PolyLine(
                    coordinates,
                    weight=3,
                    color=color,
                    opacity=0.8,
                    popup=f"Route {route.get('id', f'Route {i+1}')}"
                ).add_to(m)
            
            # Add custom start marker if enabled
            if show_start:
                folium.Marker(
                    coordinates[0],
                    popup='Start',
                    icon=folium.Icon(color=colors[i % len(colors)], icon='flag'),
                    tooltip=f"Start of Route {route.get('id', f'Route {i+1}')}"
                ).add_to(m)
            
            # Add custom end marker if enabled and route has more than one point
            if show_end and len(coordinates) > 1:
                folium.Marker(
                    coordinates[-1],
                    popup='End',
                    icon=folium.Icon(color=colors[i % len(colors)], icon='flag', prefix='fa'),
                    tooltip=f"End of Route {route.get('id', f'Route {i+1}')}"
                ).add_to(m)
                
        except Exception as e:
            print(f"Warning: Error processing route {route.get('id', i)}: {str(e)}")
            continue
    
    # Save the map
    try:
        m.save('routes_map.html')
    except Exception as e:
        print(f"Warning: Could not save map: {str(e)}")
    
    return m

In [3]:
def load_and_plot_routes(file_path, show_start=True, show_end=True, show_route_lines=True):
    """
    Load JSON file and create map with configurable features.
    
    Args:
        file_path (str): Path to JSON file
        show_start (bool): Whether to show start markers
        show_end (bool): Whether to show end markers
        show_route_lines (bool): Whether to show route lines
    """
    try:
        with open(file_path, 'r') as f:
            json_data = json.load(f)
        
        return plot_routes(json_data, show_start, show_end, show_route_lines)
    except json.JSONDecodeError as e:
        print(f"Error: Invalid JSON file: {str(e)}")
        return None
    except FileNotFoundError:
        print(f"Error: File not found: {file_path}")
        return None
    except Exception as e:
        print(f"Error: Unexpected error occurred: {str(e)}")
        return None

In [4]:
# Show only route lines, no markers
file_path = 'bike-routes-2024-01-01.json'
map_obj = load_and_plot_routes(file_path, show_start=False, show_end=False)